# Seq2Seq Translation with LSTM  
### English to French Machine Translation

**Bao gồm:**
- Chuẩn bị dữ liệu (raw → processed)
- Tạo từ vựng (src + tgt)
- Dataset + DataLoader
- Encoder–Decoder LSTM
- Training loop (teacher forcing)
- Inference (translate function)
- Evaluation (BLEU score)
- 5 ví dụ dịch + phân tích lỗi

---

### Import thư viện 

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from nltk.translate.bleu_score import sentence_bleu
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
import random

In [2]:
import sys
print(sys.executable)

/home/hanhut/miniconda3/envs/nlp/bin/python3.10


# 1. Load dataset

In [3]:
# def load_parallel(en_path, fr_path):
#     en_lines = open(en_path, 'r', encoding='utf-8').read().strip().split('\n')
#     fr_lines = open(fr_path, 'r', encoding='utf-8').read().strip().split('\n')

#     assert len(en_lines) == len(fr_lines), "Số dòng 2 file không khớp nhau!"

#     pairs = []

#     for en, fr in zip(en_lines, fr_lines):
#         pairs.append((en.lower(), fr.lower()))
    
#     return pairs

# train_pairs = load_parallel('./data/raw/train.en', './data/raw/train.fr')
# val_pairs = load_parallel('./data/raw/val.en', './data/raw/val.fr')


# 2. Build Vocabulary

In [4]:
# SPECIL_TOKENS = ['<pad>', '<sos>', '<eos>', '<unk>']

# def tokenize(sentence):
#     return sentence.strip().split()

# def yield_tokens(pairs):
#     for en, fr in pairs:
#         for token in tokenize(en):
#             yield token
#         for token in tokenize(fr):
#             yield token

# def build_vocab(iterator):
#     vocab = build_vocab_from_iterator(iterator, 
#                                     min_freq=2,
#                                     specials=SPECIL_TOKENS,
#                                     special_first=True,
#                                     max_tokens=10000)
#     vocab.set_default_index(vocab['<unk>'])
#     return vocab

# from torchtext.vocab import build_vocab_from_iterator



In [5]:
# # === Cell: Chuẩn bị dữ liệu (tokenize, build vocab, dataset, dataloader) ===
# # Yêu cầu: raw files dạng data/raw/train.en, data/raw/train.fr, valid.en/valid.fr, test.en/test.fr
# # Sử dụng spaCy tokenizer; vocab giới hạn 10000; thêm tokens: <unk>, <pad>, <sos>, <eos>.
# # Nếu torchtext khả dụng -> dùng build_vocab_from_iterator; nếu không -> fallback bằng Counter.

# import warnings
# from collections import Counter
# from torch.nn.utils.rnn import pad_sequence

# # --- spaCy tokenizer (bắt buộc đã cài en_core_web_sm và fr_core_news_sm) ---
# import spacy
# en_nlp = spacy.load("en_core_web_sm")
# fr_nlp = spacy.load("fr_core_news_sm")

# def tokenize_en(text):
#     return [tok.text.lower() for tok in en_nlp(text)]

# def tokenize_fr(text):
#     return [tok.text.lower() for tok in fr_nlp(text)]

# # --- Try to import torchtext build_vocab_from_iterator; fallback if not present ---
# USE_TORCHTEXT = False
# try:
#     from torchtext.vocab import build_vocab_from_iterator
#     USE_TORCHTEXT = True
# except Exception as e:
#     warnings.warn("torchtext unavailable or failed to load; falling back to Counter-based vocab builder.\n" + str(e))
#     USE_TORCHTEXT = False

# # --- Special tokens and options ---
# PAD_TOKEN = "<pad>"
# SOS_TOKEN = "<sos>"
# EOS_TOKEN = "<eos>"
# UNK_TOKEN = "<unk>"
# SPECIALS = [UNK_TOKEN, PAD_TOKEN, SOS_TOKEN, EOS_TOKEN]
# MAX_VOCAB = 10000
# MIN_FREQ = 1   # you can set to 2 to filter very rare tokens if desired

# # --- Helpers to read parallel files (EN/FR) ---
# def load_parallel(en_path, fr_path):
#     with open(en_path, encoding="utf8") as f:
#         en_lines = [l.strip() for l in f if l.strip()]
#     with open(fr_path, encoding="utf8") as f:
#         fr_lines = [l.strip() for l in f if l.strip()]
#     assert len(en_lines) == len(fr_lines), f"Mismatch lines: {len(en_lines)} != {len(fr_lines)}"
#     return list(zip(en_lines, fr_lines))

# # Example usage:
# # train_pairs = load_parallel("data/raw/train.en", "data/raw/train.fr")

# # --- Build vocab: two implementations ---
# if USE_TORCHTEXT:
#     # Using torchtext.build_vocab_from_iterator (preferred when available)
#     def yield_tokens_from_pairs(pairs, lang="en"):
#         if lang == "en":
#             for en, fr in pairs:
#                 yield tokenize_en(en)
#         else:
#             for en, fr in pairs:
#                 yield tokenize_fr(fr)

#     def build_vocab_torchtext(pairs, lang="en", max_tokens=MAX_VOCAB, min_freq=MIN_FREQ):
#         iterator = yield_tokens_from_pairs(pairs, lang=lang)
#         vocab = build_vocab_from_iterator(iterator,
#                                           specials=SPECIALS,
#                                           special_first=True,
#                                           max_tokens=max_tokens)
#         vocab.set_default_index(vocab[UNK_TOKEN])
#         return vocab

# else:
#     # Fallback: use Counter and build dict
#     def build_vocab_counter(pairs, lang="en", max_tokens=MAX_VOCAB, min_freq=MIN_FREQ):
#         counter = Counter()
#         if lang == "en":
#             for en, fr in pairs:
#                 counter.update(tokenize_en(en))
#         else:
#             for en, fr in pairs:
#                 counter.update(tokenize_fr(fr))
#         # keep most common up to max_tokens - len(SPECIALS)
#         most_common = [w for w, c in counter.most_common(max_tokens - len(SPECIALS)) if c >= min_freq]
#         vocab_list = SPECIALS + most_common
#         stoi = {w: i for i, w in enumerate(vocab_list)}
#         # helper object to mimic torchtext vocab API minimally
#         class VocabLike:
#             def __init__(self, stoi, itos):
#                 self.stoi = stoi
#                 self.itos = itos
#             def __len__(self):
#                 return len(self.itos)
#             def __getitem__(self, token):
#                 return self.stoi.get(token, self.stoi[UNK_TOKEN])
#             def get_itos(self):
#                 return self.itos
#             def get_stoi(self):
#                 return self.stoi
#             def set_default_index(self, idx):
#                 pass
#         itos = vocab_list
#         return VocabLike(stoi, itos)

# # --- Encode / numericalize helpers (works for both vocab types) ---
# def tokens_to_ids(tokens, vocab, specials_map=None):
#     # vocab could be torchtext.Vocab or our VocabLike
#     ids = []
#     if USE_TORCHTEXT:
#         for t in tokens:
#             ids.append(vocab[t])
#     else:
#         for t in tokens:
#             ids.append(vocab.stoi.get(t, vocab.stoi[UNK_TOKEN]))
#     return ids

# def encode_sentence_en(text, vocab):
#     toks = tokenize_en(text)
#     ids = [ (vocab[SOS_TOKEN] if USE_TORCHTEXT else vocab.stoi[SOS_TOKEN]) ]
#     ids += tokens_to_ids(toks, vocab)
#     ids += [ (vocab[EOS_TOKEN] if USE_TORCHTEXT else vocab.stoi[EOS_TOKEN]) ]
#     return torch.tensor(ids, dtype=torch.long)

# def encode_sentence_fr(text, vocab):
#     toks = tokenize_fr(text)
#     ids = [ (vocab[SOS_TOKEN] if USE_TORCHTEXT else vocab.stoi[SOS_TOKEN]) ]
#     ids += tokens_to_ids(toks, vocab)
#     ids += [ (vocab[EOS_TOKEN] if USE_TORCHTEXT else vocab.stoi[EOS_TOKEN]) ]
#     return torch.tensor(ids, dtype=torch.long)

# # --- Dataset + collate_fn (pad + sort by length descending + return lengths) ---
# class ParallelDataset(Dataset):
#     def __init__(self, pairs, src_vocab, tgt_vocab):
#         self.pairs = pairs
#         self.src_vocab = src_vocab
#         self.tgt_vocab = tgt_vocab

#     def __len__(self):
#         return len(self.pairs)

#     def __getitem__(self, idx):
#         en, fr = self.pairs[idx]
#         src_ids = encode_sentence_en(en, self.src_vocab)
#         tgt_ids = encode_sentence_fr(fr, self.tgt_vocab)
#         return src_ids, tgt_ids

# def collate_fn(batch):
#     # batch: list of (src_tensor, tgt_tensor)
#     src_list, tgt_list = zip(*batch)
#     # sort by src length desc (required for pack_padded_sequence)
#     sorted_pairs = sorted(zip(src_list, tgt_list), key=lambda x: -x[0].size(0))
#     src_list, tgt_list = zip(*sorted_pairs)
#     src_padded = pad_sequence(src_list, batch_first=True, padding_value=(vocab_en[PAD_TOKEN] if USE_TORCHTEXT else vocab_en.stoi[PAD_TOKEN]))
#     tgt_padded = pad_sequence(tgt_list, batch_first=True, padding_value=(vocab_fr[PAD_TOKEN] if USE_TORCHTEXT else vocab_fr.stoi[PAD_TOKEN]))
#     src_lengths = torch.tensor([s.size(0) for s in src_list], dtype=torch.long)
#     tgt_lengths = torch.tensor([t.size(0) for t in tgt_list], dtype=torch.long)
#     return src_padded, tgt_padded, src_lengths, tgt_lengths

# # === Usage example ===
# # load data
# train_pairs = load_parallel("data/raw/train.en", "data/raw/train.fr")
# valid_pairs = load_parallel("data/raw/val.en", "data/raw/val.fr")
# test_pairs  = load_parallel("data/raw/test_2016_flickr.en",  "data/raw/test_2016_flickr.fr")

# # build vocabs (prefer torchtext if available)
# if USE_TORCHTEXT:
#     vocab_en = build_vocab_torchtext(train_pairs, lang="en", max_tokens=MAX_VOCAB, min_freq=MIN_FREQ)
#     vocab_fr = build_vocab_torchtext(train_pairs, lang="fr", max_tokens=MAX_VOCAB, min_freq=MIN_FREQ)
# else:
#     vocab_en = build_vocab_counter(train_pairs, lang="en", max_tokens=MAX_VOCAB, min_freq=MIN_FREQ)
#     vocab_fr = build_vocab_counter(train_pairs, lang="fr", max_tokens=MAX_VOCAB, min_freq=MIN_FREQ)

# print("Vocab sizes -> EN:", len(vocab_en) if USE_TORCHTEXT else len(vocab_en.itos),
#       " FR:", len(vocab_fr) if USE_TORCHTEXT else len(vocab_fr.itos))

# # create dataloaders
# batch_size = 64
# train_ds = ParallelDataset(train_pairs, vocab_en, vocab_fr)
# valid_ds = ParallelDataset(valid_pairs, vocab_en, vocab_fr)
# test_ds  = ParallelDataset(test_pairs,  vocab_en, vocab_fr)

# train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
# valid_loader = DataLoader(valid_ds, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
# test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# print('DataLoaders ready. Example batch shapes:')
# for src_batch, tgt_batch, src_lens, tgt_lens in train_loader:
#     print(src_batch.shape, tgt_batch.shape, src_lens.shape, tgt_lens.shape)
#     break


In [28]:
from processed_data import (build_vocab, load_parallel, ParallelDataset, make_collate_fn, encode_sentence_en, encode_sentence_fr, save_vocab, load_vocab, save_dataset_pytorch, load_dataset_pytorch)

In [7]:
# train_pairs = load_parallel('./data/raw/train.en', './data/raw/train.fr')
# vocab_en = build_vocab(train_pairs, lang="en")
# vocab_fr = build_vocab(train_pairs, lang="fr")

# train_ds = ParallelDataset(train_pairs, vocab_en, vocab_fr)



In [8]:
# save_vocab(vocab_en, './data/processed/vocab_en.pkl')
# save_vocab(vocab_fr, './data/processed/vocab_fr.pkl')

In [9]:
# valid_pairs = load_parallel('./data/raw/val.en', './data/raw/val.fr')
# test_pairs = load_parallel('./data/raw/test_2016_flickr.en', './data/raw/test_2016_flickr.fr')

# valid_ds = ParallelDataset(valid_pairs, vocab_en, vocab_fr)
# test_ds = ParallelDataset(test_pairs, vocab_en, vocab_fr)

# save_dataset_pytorch(train_ds, './data/processed/train_dataset.pt')
# save_dataset_pytorch(valid_ds, './data/processed/valid_dataset.pt')
# save_dataset_pytorch(test_ds, './data/processed/test_dataset.pt')

In [29]:
vocab_en = load_vocab('./data/processed/vocab_en.pkl')
vocab_fr = load_vocab('./data/processed/vocab_fr.pkl')

train_ds = load_dataset_pytorch('./data/processed/train_dataset.pt')
valid_ds = load_dataset_pytorch('./data/processed/valid_dataset.pt')
test_ds = load_dataset_pytorch('./data/processed/test_dataset.pt')

collate_fn = make_collate_fn(vocab_en, vocab_fr)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_ds, batch_size=64, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_ds, batch_size=64, shuffle=False, collate_fn=collate_fn)

# 3. LSTM Model

In [30]:
class LSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.W_i = nn.Linear(input_size, hidden_size, bias=True)
        self.U_i = nn.Linear(hidden_size, hidden_size, bias=True)
        self.W_f = nn.Linear(input_size, hidden_size, bias=True)
        self.U_f = nn.Linear(hidden_size, hidden_size, bias=True)
        self.W_o = nn.Linear(input_size, hidden_size, bias=True)
        self.U_o = nn.Linear(hidden_size, hidden_size, bias=True)
        self.W_c = nn.Linear(input_size, hidden_size, bias=True)
        self.U_c = nn.Linear(hidden_size, hidden_size,  bias=True)

    def forward(self, x, h_prev, c_prev):
        i_t = torch.sigmoid(self.W_i(x) + self.U_i(h_prev))
        f_t = torch.sigmoid(self.W_f(x) + self.U_f(h_prev))
        o_t = torch.sigmoid(self.W_o(x) + self.U_o(h_prev))
        c_tilde_t = torch.tanh(self.W_c(x) + self.U_c(h_prev))
        c_t = f_t * c_prev + i_t * c_tilde_t
        h_t = o_t * torch.tanh(c_t)
        return h_t, c_t

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=2):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cells = nn.ModuleList([LSTMCell(input_size if i == 0 else hidden_size, hidden_size) for i in range(num_layers)])

    def forward(self, x, h_0=None, c_0=None):
        batch_size, seq_len, _ = x.size()
        if h_0 is None:
            h_0 = [torch.zeros(batch_size, self.hidden_size, device=x.device) for _ in range(self.num_layers)]
        if c_0 is None:
            c_0 = [torch.zeros(batch_size, self.hidden_size, device=x.device) for _ in range(self.num_layers)]

        h_n = []
        c_n = []
        outputs = []

        for t in range(seq_len):
            x_t = x[:, t, :]
            for layer in range(self.num_layers):
                h_prev = h_0[layer]
                c_prev = c_0[layer]
                h_t, c_t = self.cells[layer](x_t, h_prev, c_prev)
                h_0[layer] = h_t
                c_0[layer] = c_t
                x_t = h_t
            outputs.append(h_t.unsqueeze(1))

        outputs = torch.cat(outputs, dim=1)
        h_n = h_0
        c_n = c_0

        return outputs, (h_n, c_n)

class Encoder(nn.Module):
    def __init__(self, input_size, embed_size, hidden_size, num_layers=2):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_size, embed_size)
        self.lstm = LSTM(embed_size, hidden_size, num_layers)
        self.hidden_size = hidden_size
    
    def forward(self, x):
        embedded = self.embedding(x)
        outputs, (h_n, c_n) = self.lstm(embedded)
        return outputs, (h_n, c_n)

class Decoder(nn.Module):
    def __init__(self, output_size, embed_size, hidden_size, num_layers=2):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(output_size, embed_size)
        self.lstm = LSTM(embed_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, x, h_0, c_0):
        embedded = self.embedding(x)
        outputs, (h_n, c_n) = self.lstm(embedded, h_0, c_0)
        logits = self.fc(outputs)
        probs = F.softmax(logits, dim=-1)
        return probs, (h_n, c_n)
    
    def forward_step(self, y_prev, h, c):
        # y_prev: (batch,)
        y_prev = y_prev.unsqueeze(1)         # (batch, 1)
        embedded = self.embedding(y_prev)    # (batch, 1, embed)
        
        outputs, (h, c) = self.lstm(embedded, h, c)
        logits = self.fc(outputs[:, -1, :])  # lấy token cuối
        probs = F.softmax(logits, dim=-1)

        return probs, h, c
    
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, sos_id, eos_id):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.sos_id = sos_id
        self.eos_id = eos_id
    
    def forward(self, src, tgt, teacher_forcing_ratio=0.5):
        batch_size, tgt_len = tgt.size()
        vocab_size = self.decoder.fc.out_features

        outputs = torch.zeros(batch_size, tgt_len, vocab_size, device=src.device)

        # encoder produce h, c
        encoder_outputs, (h, c) = self.encoder(src)

        # start token
        y_prev = torch.full((batch_size,), self.sos_id, device=src.device)

        for t in range(tgt_len):
            probs, h, c = self.decoder.forward_step(y_prev, h, c)
            outputs[:, t, :] = probs

            # teacher forcing
            use_tf = torch.rand(1).item() < teacher_forcing_ratio
            y_prev = tgt[:, t] if use_tf else probs.argmax(dim=-1)

        return outputs



In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

encoder = Encoder(input_size=len(vocab_en), embed_size=256, hidden_size=512, num_layers=2)
decoder = Decoder(output_size=len(vocab_fr), embed_size=256, hidden_size=512, num_layers=2)
seq2seq_model = Seq2Seq(encoder, decoder, sos_id=vocab_fr['<sos>'], eos_id=vocab_fr['<eos>']).to(device)

Using device: cpu


# 4. Model Training

In [13]:
pad_idx = vocab_fr['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
optimizer = torch.optim.Adam(seq2seq_model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=1, factor=0.5
)


In [14]:
def train_model(model, train_loader, valid_loader, num_epochs=20, teacher_forcing_ratio=0.5):
    best_val_loss = float("inf")
    patience = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0

        for src, tgt, src_lens, tgt_lens in train_loader:
            
            src = src.to(device)
            tgt = tgt.to(device)

            optimizer.zero_grad()

            outputs = model(src, tgt, teacher_forcing_ratio)   # (B, T, V)
            outputs = outputs[:, :-1, :].reshape(-1, outputs.size(-1))
            tgt_gold = tgt[:, 1:].reshape(-1)

            loss = criterion(outputs, tgt_gold)
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()
            train_loss += loss.item()

        # ---- VALIDATION ----
        model.eval()
        val_loss = 0
        
        with torch.no_grad():
            for src, tgt, _, _ in valid_loader:
                
                src = src.to(device)
                tgt = tgt.to(device)

                outputs = model(src, tgt, 0)
                outputs = outputs[:, :-1, :].reshape(-1, outputs.size(-1))
                tgt_gold = tgt[:, 1:].reshape(-1)

                loss = criterion(outputs, tgt_gold)
                val_loss += loss.item()

        scheduler.step(val_loss)

        print(f"Epoch {epoch+1} | Train: {train_loss:.4f} | Val: {val_loss:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience = 0
            torch.save(model.state_dict(), "best_model.pt")
            print("  → Saved best model")
        else:
            patience += 1
            if patience >= 3:
                print("Early stopping triggered!")
                break

In [ ]:
train_model(seq2seq_model, train_loader, valid_loader, num_epochs=20, teacher_forcing_ratio=0.5)

Epoch 1 | Train: 3924.4721 | Val: 138.0564
  → Saved best model


In [ ]:
print(torch.cuda.is_available())
print(torch.__version__)

In [32]:
def translate(sentence, model, vocab_en, vocab_fr, max_len=50):
    model.eval()

    # tokenize
    tokens = sentence.lower().split()
    ids = [vocab_en.stoi.get(tok, vocab_en.stoi["<unk>"]) for tok in tokens]
    src = torch.tensor(ids).unsqueeze(0).to(next(model.parameters()).device)

    # encode
    _, (h, c) = model.encoder(src)

    # decode từng bước
    y_prev = torch.tensor([vocab_fr.stoi["<sos>"]], device=src.device)
    result_ids = []

    for _ in range(max_len):
        probs, h, c = model.decoder.forward_step(y_prev, h, c)
        y_prev = probs.argmax(dim=-1)

        token_id = y_prev.item()
        if token_id == vocab_fr.stoi["<eos>"]:
            break
        
        result_ids.append(token_id)

    # convert id -> word bằng vocab_fr.itos
    words = [vocab_fr.itos[i] for i in result_ids]
    return " ".join(words)

In [ ]:

def compute_bleu(test_data):
    bleu_scores = []

    for eng, fr in test_data:
        pred = translate(eng)

        reference = [fr.split()]   # gold
        candidate = pred.split()   # prediction

        bleu = sentence_bleu(reference, candidate)
        bleu_scores.append(bleu)

    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    return avg_bleu

In [33]:
def decode_tensor(tensor_ids, vocab):
    # vocab.itos là list: index -> token
    tokens = []
    for idx in tensor_ids.tolist():
        if idx < len(vocab.itos):
            tokens.append(vocab.itos[idx])
    # bỏ pad/sos/eos
    tokens = [t for t in tokens if t not in ["<pad>", "<sos>", "<eos>"]]
    return " ".join(tokens)

def show_examples(test_ds, model, vocab_en, vocab_fr, n=5):
    for i in range(n):
        src_tensor, tgt_tensor = test_ds[i]

        eng = decode_tensor(src_tensor, vocab_en)
        fr  = decode_tensor(tgt_tensor, vocab_fr)
        pred = translate(eng, model, vocab_en, vocab_fr)

        print(f"\n[Example {i+1}]")
        print(f"EN:   {eng}")
        print(f"FR:   {fr}")
        print(f"PRED: {pred}")


In [34]:
seq2seq_model.load_state_dict(torch.load("best_model.pt", map_location=device))
seq2seq_model.eval()
show_examples(test_ds, seq2seq_model, vocab_en, vocab_fr, n=5)




[Example 1]
EN:   A man in an orange hat starring at something .
FR:   Un homme avec un chapeau orange regardant quelque chose .
PRED: Un homme un un un un un un un un un .

[Example 2]
EN:   A Boston Terrier is running on lush green grass in front of a white fence .
FR:   Un terrier de Boston court sur l' herbe verdoyante devant une clôture blanche .
PRED: Un homme homme un un un un un un un un un un un . .

[Example 3]
EN:   A girl in karate uniform breaking a stick with a front kick .
FR:   Une fille en tenue de karaté brisant un bâton avec un coup de pied .
PRED: Un homme homme un un un un un un un un un un . .

[Example 4]
EN:   Five people wearing winter jackets and helmets stand in the snow , with <unk> in the background .
FR:   Cinq personnes avec des vestes d' hiver et des casques sont debout dans la neige , avec des <unk> en arrière-plan .
PRED: Un homme homme un un un un un un un un un un un un un . .

[Example 5]
EN:   People are fixing the roof of a house .
FR:   Des gens

In [ ]:
print(vocab_en.__dict__)
